In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
%matplotlib inline
# import xgboost as xgb

pd.set_option('display.max_columns', 500)


from sklearn import model_selection, preprocessing
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor


import math

#A function to calculate Root Mean Squared Logarithmic Error (RMSLE)
def rmsle(y_pred, y_test) : 
    assert len(y_test) == len(y_pred)
    return np.sqrt(np.mean((np.log(1+y_pred) - np.log(1+y_test))**2))

C:\Users\danao\AppData\Local\Continuum\anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

#Convert price to log
ylog_train_all = np.log1p(df_train['price_doc'].values)
id_test = df_test['id']

df_train.drop(['id', 'price_doc'], axis=1, inplace=True)
df_test.drop(['id'], axis=1, inplace=True)

num_train = len(df_train)
input_df = pd.concat([df_train, df_test])

input_df.replace(['yes','no'],[1,0],inplace=True)


In [8]:
input_df_nona = input_df.select_dtypes(exclude=['object'])
input_df_nona['timestamp'] = pd.to_datetime(input_df['timestamp'])


C:\Users\danao\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# working on time

In [9]:
input_df_nona['year'] = input_df_nona.timestamp.dt.year
input_df_nona['month'] = input_df_nona.timestamp.dt.month
input_df_nona['week'] = input_df_nona.timestamp.dt.week
input_df_nona['day'] = input_df_nona.timestamp.dt.day
input_df_nona['dow'] = input_df_nona.timestamp.dt.dayofweek



C:\Users\danao\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\danao\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
# Add month-year
month_year = (input_df_nona.timestamp.dt.month + input_df_nona.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
input_df_nona['month_year_cnt'] = month_year.map(month_year_cnt_map)

In [11]:
# Add week-year count
week_year = (input_df_nona.timestamp.dt.weekofyear + input_df_nona.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
input_df_nona['week_year_cnt'] = week_year.map(week_year_cnt_map)
input_df_nona['runningmonth']=(input_df_nona.timestamp.dt.year - 2011 ) * 12 + (input_df_nona.timestamp.dt.month - 8 ) + 1
input_df_nona.drop(['timestamp'], axis=1, inplace=True)
input_df_nona.head(1)

full_sq  life_sq  floor  max_floor  material  build_year  num_room  \
0     43.0     27.0    4.0        NaN       NaN         NaN       NaN   

   kitch_sq  state     area_m  raion_popul  green_zone_part  indust_part  \
0       NaN    NaN  6407578.1       155572         0.189727      0.00007   

   children_preschool  preschool_quota  preschool_education_centers_raion  \
0                9576           5001.0                                  5   

   children_school  school_quota  school_education_centers_raion  \
0            10309       11065.0                               5   

   school_education_centers_top_20_raion  hospital_beds_raion  \
0                                      0                240.0   

   healthcare_centers_raion  university_top_20_raion  sport_objects_raion  \
0                         1                        0                    7   

   additional_education_raion  culture_objects_top_25  \
0                           3                       0   

   culture_objects_top_25_raion  shopping_centers_raion  office_raion  \
0                             0                      16             1   

   thermal_power_plant_raion  incineration_raion  oil_chemistry_raion  \
0                          0                   0                    0   

   radiation_raion  railroad_terminal_raion  big_market_raion  \
0                0                        0                 0   

   nuclear_reactor_raion  detention_facility_raion  full_all  male_f  \
0                      0                         0     86206   40477   

   female_f  young_all  young_male  young_female  work_all  work_male  \
0     45729      21154       11007         10147     98207      52277   

   work_female  ekder_all  ekder_male  ekder_female  0_6_all  0_6_male  \
0        45930      36211       10580         25631     9576      4899   

   0_6_female  7_14_all  7_14_male  7_14_female  0_17_all  0_17_male  \
0        4677     10309       5463         4846     23603      12286   

   0_17_female  16_29_all  16_29_male  16_29_female  0_13_all  0_13_male  \
0        11317      17508        9425          8083     18654       9709   

   0_13_female  raion_build_count_with_material_info  build_count_block  \
0         8945                                 211.0               25.0   

   build_count_wood  build_count_frame  build_count_brick  \
0               0.0                0.0                0.0   

   build_count_monolith  build_count_panel  build_count_foam  \
0                   2.0              184.0               0.0   

   build_count_slag  build_count_mix  raion_build_count_with_builddate_info  \
0               0.0              0.0                                  211.0   

   build_count_before_1920  build_count_1921-1945  build_count_1946-1970  \
0                      0.0                    0.0                    0.0   

   build_count_1971-1995  build_count_after_1995  ID_metro  metro_min_avto  \
0                  206.0                     5.0         1        2.590241   

   metro_km_avto  metro_min_walk  metro_km_walk  kindergarten_km  school_km  \
0        1.13126       13.575119        1.13126           0.1457   0.177975   

    park_km  green_zone_km  industrial_km  water_treatment_km  cemetery_km  \
0  2.158587       0.600973       1.080934            23.68346     1.804127   

   incineration_km  railroad_station_walk_km  railroad_station_walk_min  \
0         3.633334                  5.419893                  65.038716   

   ID_railroad_station_walk  railroad_station_avto_km  \
0                       1.0                  5.419893   

   railroad_station_avto_min  ID_railroad_station_avto  \
0                   6.905893                         1   

   public_transport_station_km  public_transport_station_min_walk  water_km  \
0                     0.274985                           3.299822  0.992631   

   water_1line   mkad_km     ttk_km  sadovoe_km  bulvar_ring_km  kremlin_km  \
0            0  1.422391  10.918587  

In [13]:
input_df_nona['rel_floor']     = input_df_nona['floor']       / input_df_nona['max_floor'].astype(float)
input_df_nona['rel_kitch_sq']  = input_df_nona['kitch_sq']    / input_df_nona['life_sq'].astype(float)
input_df_nona['rel_life_sq']   = input_df_nona['life_sq']     / input_df_nona['full_sq'].astype(float)

# working on objects 

In [14]:
df_obj = input_df.select_dtypes(include=['object']).copy()
df_obj.head()

product_type           sub_area    ecology
0   Investment           Bibirevo       good
1   Investment  Nagatinskij Zaton  excellent
2   Investment     Tekstil'shhiki       poor
3   Investment             Mitino       good
4   Investment          Basmannoe  excellent

In [15]:
input_df_nona['ecology'] = df_obj.ecology.replace(['no data','poor','satisfactory','good','excellent'], [0,1,2,3,4])

In [16]:
input_df_nona['product_type'] = df_obj.product_type.replace(['Investment','OwnerOccupier'], [0,1])

In [17]:
input_df_nona.head(1)

full_sq  life_sq  floor  max_floor  material  build_year  num_room  \
0     43.0     27.0    4.0        NaN       NaN         NaN       NaN   

   kitch_sq  state     area_m  raion_popul  green_zone_part  indust_part  \
0       NaN    NaN  6407578.1       155572         0.189727      0.00007   

   children_preschool  preschool_quota  preschool_education_centers_raion  \
0                9576           5001.0                                  5   

   children_school  school_quota  school_education_centers_raion  \
0            10309       11065.0                               5   

   school_education_centers_top_20_raion  hospital_beds_raion  \
0                                      0                240.0   

   healthcare_centers_raion  university_top_20_raion  sport_objects_raion  \
0                         1                        0                    7   

   additional_education_raion  culture_objects_top_25  \
0                           3                       0   

   culture_objects_top_25_raion  shopping_centers_raion  office_raion  \
0                             0                      16             1   

   thermal_power_plant_raion  incineration_raion  oil_chemistry_raion  \
0                          0                   0                    0   

   radiation_raion  railroad_terminal_raion  big_market_raion  \
0                0                        0                 0   

   nuclear_reactor_raion  detention_facility_raion  full_all  male_f  \
0                      0                         0     86206   40477   

   female_f  young_all  young_male  young_female  work_all  work_male  \
0     45729      21154       11007         10147     98207      52277   

   work_female  ekder_all  ekder_male  ekder_female  0_6_all  0_6_male  \
0        45930      36211       10580         25631     9576      4899   

   0_6_female  7_14_all  7_14_male  7_14_female  0_17_all  0_17_male  \
0        4677     10309       5463         4846     23603      12286   

   0_17_female  16_29_all  16_29_male  16_29_female  0_13_all  0_13_male  \
0        11317      17508        9425          8083     18654       9709   

   0_13_female  raion_build_count_with_material_info  build_count_block  \
0         8945                                 211.0               25.0   

   build_count_wood  build_count_frame  build_count_brick  \
0               0.0                0.0                0.0   

   build_count_monolith  build_count_panel  build_count_foam  \
0                   2.0              184.0               0.0   

   build_count_slag  build_count_mix  raion_build_count_with_builddate_info  \
0               0.0              0.0                                  211.0   

   build_count_before_1920  build_count_1921-1945  build_count_1946-1970  \
0                      0.0                    0.0                    0.0   

   build_count_1971-1995  build_count_after_1995  ID_metro  metro_min_avto  \
0                  206.0                     5.0         1        2.590241   

   metro_km_avto  metro_min_walk  metro_km_walk  kindergarten_km  school_km  \
0        1.13126       13.575119        1.13126           0.1457   0.177975   

    park_km  green_zone_km  industrial_km  water_treatment_km  cemetery_km  \
0  2.158587       0.600973       1.080934            23.68346     1.804127   

   incineration_km  railroad_station_walk_km  railroad_station_walk_min  \
0         3.633334                  5.419893                  65.038716   

   ID_railroad_station_walk  railroad_station_avto_km  \
0                       1.0                  5.419893   

   railroad_station_avto_min  ID_railroad_station_avto  \
0                   6.905893                         1   

   public_transport_station_km  public_transport_station_min_walk  water_km  \
0                     0.274985                           3.299822  0.992631   

   water_1line   mkad_km     ttk_km  sadovoe_km  bulvar_ring_km  kremlin_km  \
0            0  1.422391  10.918587  

In [18]:
input_df_nona['sub_area_fac'] = pd.factorize(df_obj['sub_area'])[0]

In [19]:
input_df_nona.head(1)

full_sq  life_sq  floor  max_floor  material  build_year  num_room  \
0     43.0     27.0    4.0        NaN       NaN         NaN       NaN   

   kitch_sq  state     area_m  raion_popul  green_zone_part  indust_part  \
0       NaN    NaN  6407578.1       155572         0.189727      0.00007   

   children_preschool  preschool_quota  preschool_education_centers_raion  \
0                9576           5001.0                                  5   

   children_school  school_quota  school_education_centers_raion  \
0            10309       11065.0                               5   

   school_education_centers_top_20_raion  hospital_beds_raion  \
0                                      0                240.0   

   healthcare_centers_raion  university_top_20_raion  sport_objects_raion  \
0                         1                        0                    7   

   additional_education_raion  culture_objects_top_25  \
0                           3                       0   

   culture_objects_top_25_raion  shopping_centers_raion  office_raion  \
0                             0                      16             1   

   thermal_power_plant_raion  incineration_raion  oil_chemistry_raion  \
0                          0                   0                    0   

   radiation_raion  railroad_terminal_raion  big_market_raion  \
0                0                        0                 0   

   nuclear_reactor_raion  detention_facility_raion  full_all  male_f  \
0                      0                         0     86206   40477   

   female_f  young_all  young_male  young_female  work_all  work_male  \
0     45729      21154       11007         10147     98207      52277   

   work_female  ekder_all  ekder_male  ekder_female  0_6_all  0_6_male  \
0        45930      36211       10580         25631     9576      4899   

   0_6_female  7_14_all  7_14_male  7_14_female  0_17_all  0_17_male  \
0        4677     10309       5463         4846     23603      12286   

   0_17_female  16_29_all  16_29_male  16_29_female  0_13_all  0_13_male  \
0        11317      17508        9425          8083     18654       9709   

   0_13_female  raion_build_count_with_material_info  build_count_block  \
0         8945                                 211.0               25.0   

   build_count_wood  build_count_frame  build_count_brick  \
0               0.0                0.0                0.0   

   build_count_monolith  build_count_panel  build_count_foam  \
0                   2.0              184.0               0.0   

   build_count_slag  build_count_mix  raion_build_count_with_builddate_info  \
0               0.0              0.0                                  211.0   

   build_count_before_1920  build_count_1921-1945  build_count_1946-1970  \
0                      0.0                    0.0                    0.0   

   build_count_1971-1995  build_count_after_1995  ID_metro  metro_min_avto  \
0                  206.0                     5.0         1        2.590241   

   metro_km_avto  metro_min_walk  metro_km_walk  kindergarten_km  school_km  \
0        1.13126       13.575119        1.13126           0.1457   0.177975   

    park_km  green_zone_km  industrial_km  water_treatment_km  cemetery_km  \
0  2.158587       0.600973       1.080934            23.68346     1.804127   

   incineration_km  railroad_station_walk_km  railroad_station_walk_min  \
0         3.633334                  5.419893                  65.038716   

   ID_railroad_station_walk  railroad_station_avto_km  \
0                       1.0                  5.419893   

   railroad_station_avto_min  ID_railroad_station_avto  \
0                   6.905893                         1   

   public_transport_station_km  public_transport_station_min_walk  water_km  \
0                     0.274985                           3.299822  0.992631   

   water_1line   mkad_km     ttk_km  sadovoe_km  bulvar_ring_km  kremlin_km  \
0            0  1.422391  10.918587  

# Build_year 

In [ ]:
input_df_nona['build_year_bool'] = 